In [28]:
from triagerx.model.module_factory import ModelFactory
from loguru import logger
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from triagerx.dataset.text_processor import TextProcessor
from tqdm import tqdm

tqdm.pandas()

In [29]:
torch.manual_seed(42)

In [39]:
dataset_path = "/home/mdafifal.mamun/notebooks/triagerX/data/openj9_dataset_6_7_24_most_contrib.csv"

In [40]:
df = pd.read_csv(dataset_path)

In [41]:
df.head()

,issue_number,issue_title,issue_body,issue_url,issue_state,creator,labels,assignees,component
0,2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,https://github.com/eclipse-openj9/openj9/issues/2,closed,aarongraham9,NaN,fengxue-IS,NaN
1,3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,https://github.com/eclipse-openj9/openj9/issues/3,closed,dorrab,NaN,vsebe,NaN
2,5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,https://github.com/eclipse-openj9/openj9/issues/5,closed,r30shah,NaN,anikser,NaN
3,11,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",https://github.com/eclipse-openj9/openj9/issue...,open,mgaudet,question,tajila,NaN
4,13,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,https://github.com/eclipse-openj9/openj9/issue...,closed,rservant,"enhancement, comp:build",tajila,comp:build


In [42]:
df = df.rename(columns={"assignees": "owner", "issue_body": "description"})
df = df.sort_values(by="issue_number")
df = df[df["owner"].notna()]
# df = df[df["component"].notna()]

In [43]:
df = df.sort_values(by="issue_number")
df = TextProcessor.prepare_dataframe(df, use_special_tokens=False, use_description=True, use_summary=False, component_training=False, is_openj9=True)

df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)

100%|██████████| 4510/4510 [00:00<00:00, 1088207.50it/s]
2024-06-13 15:38:49.099 | INFO     | triagerx.dataset.text_processor:prepare_dataframe:64 - Cleaning text...
100%|██████████| 4510/4510 [00:01<00:00, 3612.97it/s]
2024-06-13 15:38:50.352 | INFO     | triagerx.dataset.text_processor:prepare_dataframe:77 - Adding description...
100%|██████████| 4510/4510 [00:00<00:00, 122117.93it/s]


In [44]:
sample_threshold = 20
developers = df_train["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
df_train = df_train[df_train["owner"].isin(filtered_developers)]

train_owners = set(df_train["owner"])
test_owners = set(df_test["owner"])

unwanted = list(test_owners - train_owners)

df_test = df_test[~df_test["owner"].isin(unwanted)]

logger.info(f"Training data: {len(df_train)}, Validation data: {len(df_test)}")
logger.info(f"Number of developers: {len(df_train.owner.unique())}")

logger.info(f"Train dataset size: {len(df_train)}")
logger.info(f"Test dataset size: {len(df_test)}")

2024-06-13 15:38:50.503 | INFO     | __main__:<module>:13 - Training data: 2860, Validation data: 726
2024-06-13 15:38:50.505 | INFO     | __main__:<module>:14 - Number of developers: 41
2024-06-13 15:38:50.506 | INFO     | __main__:<module>:16 - Train dataset size: 2860
2024-06-13 15:38:50.508 | INFO     | __main__:<module>:17 - Test dataset size: 726


In [45]:
# # Generate label ids
lbl2idx = {}
idx2lbl = {}

train_owners = sorted(train_owners)

for idx, dev in enumerate(train_owners):
    lbl2idx[dev] = idx
    idx2lbl[idx] = dev

df_train["owner_id"] = df_train["owner"].apply(lambda owner: lbl2idx[owner])
df_test["owner_id"] = df_test["owner"].apply(lambda owner: lbl2idx[owner])

In [46]:
df_train = df_train[df_train["component"].notna()]
df_test = df_test[df_test["component"].notna()]

In [47]:
target_components = ["comp:vm", "comp:jvmti", "comp:jclextensions", "comp:test", "comp:build", "comp:gc"]

In [48]:
target_components = sorted(target_components)

In [49]:
comp_id2label = {}
comp_lbl2id = {}

for i, comp in enumerate(target_components):
    comp_id2label[i] = comp
    comp_lbl2id[comp] = i

In [50]:
base_transformer_models = ["microsoft/deberta-base", "roberta-base"]

In [51]:
developer_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/triagerx_ensemble_u3_42_classes_most_seed42.pt"
component_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/component_triagerx_u3_6_classes_seed42.pt"

In [52]:
logger.debug("Modeling network...")
dev_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=len(df_train.owner_id.unique()),
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=base_transformer_models,
    dropout=0.2,
    max_tokens=256,
    label_map=idx2lbl,
)

2024-06-13 15:39:54.629 | DEBUG    | __main__:<module>:1 - Modeling network...
2024-06-13 15:39:54.630 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-13 15:39:54.630 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-13 15:39:54.631 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
df_train.component.unique()

array(['comp:build', 'comp:jit', 'comp:vm', 'comp:test', 'comp:doc',
       'comp:gc', 'comp:port', 'comp:jitserver', 'comp:infra',
       'comp:jit:aot', 'comp:jclextensions', 'comp:openssl', 'comp:fips',
       'comp:jvmti'], dtype=object)

In [54]:
dev_model.load_state_dict(torch.load(developer_model_weights))

<All keys matched successfully>

In [55]:
comp_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=6,
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=["microsoft/deberta-base", "roberta-base"],
    dropout=0.2,
    max_tokens=256,
    label_map=comp_id2label,
)

2024-06-13 15:39:57.315 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-13 15:39:57.317 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-13 15:39:57.318 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
comp_model.load_state_dict(torch.load(component_model_weights))

<All keys matched successfully>

In [57]:
similarity_model = SentenceTransformer("all-mpnet-base-v2")

In [58]:
train_embeddings_path = "/home/mdafifal.mamun/notebooks/triagerX/data/openj9_embeddings/embeddings.npy"

In [59]:
encodings = similarity_model.encode(
    df_train.text.tolist(),
    show_progress_bar=True
)

import numpy as np

np.save(train_embeddings_path, encodings)

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

In [60]:
import numpy as np
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # For numerical stability
    probabilities = exp_logits / np.sum(exp_logits)
    return probabilities

In [61]:
import json
import os
from collections import defaultdict
from typing import Dict, List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from loguru import logger
from sentence_transformers import util

from triagerx.dataset.text_processor import TextProcessor
from triagerx.model.prediction_model import PredictionModel


class TriagerX:
    def __init__(
        self,
        component_prediction_model: PredictionModel,
        developer_prediction_model: PredictionModel,
        similarity_model: nn.Module,
        train_data: pd.DataFrame,
        train_embeddings: str,
        issues_path: str,
        developer_id_map: Dict[str, int],
        component_id_map: Dict[str, int],
        device: str,
    ) -> None:
        self._component_prediction_model = component_prediction_model
        self._developer_prediction_model = developer_prediction_model
        self._device = device
        self._component_prediction_model = self._component_prediction_model.to(device)
        self._developer_prediction_model = self._developer_prediction_model.to(device)
        self._similarity_model = similarity_model
        self._train_data = train_data
        self._special_tokens = TextProcessor.SPECIAL_TOKENS
        self._issues_path = issues_path
        self._all_issues = os.listdir(self._issues_path)
        self._developer2id_map = developer_id_map
        self._component2id_map = component_id_map
        self._id2developer_map = {idx: dev for dev, idx in developer_id_map.items()}
        self._id2component_map = {idx: comp for comp, idx in component_id_map.items()}
        logger.debug(f"Using device: {device}")
        logger.debug("Loading embedding for existing issues...")
        self._all_embeddings = np.load(train_embeddings)

    def get_recommendation(self, issue, k_comp, k_dev, k_rank, similarity_threshold):
        logger.debug("Prediciting components...")
        comp_prediction_score, predicted_components = self._get_predicted_components(
            issue=issue, k=k_comp
        )
        predicted_components_name = [
            self._id2component_map[idx] for idx in predicted_components
        ]
        logger.info(f"Predicted components: {predicted_components_name}")

        logger.debug("Generating developer recommendation...")
        dev_prediction_score, predicted_devs = self._get_recommendation_from_dev_model(
            issue=issue, k=k_rank
        )
        predicted_developers_name = [
            self._id2developer_map[idx] for idx in predicted_devs
        ]
        logger.info(f"Recommended developers: {predicted_developers_name}")

        logger.debug("Generating recommendation by similarity...")
        dev_predictions_by_similarity = self._get_recommendation_by_similarity(
            issue,
            predicted_components,
            k_dev=k_rank,
            k_issue=k_rank,
            similarity_threshold=similarity_threshold,
        )
        similar_issue_devs = [dev_sim for dev_sim, _ in dev_predictions_by_similarity]
        logger.info(f"Recommended developers by issue similarity: {similar_issue_devs}")

        logger.debug("Aggregating ranking...")
        rank_lists = [predicted_developers_name, similar_issue_devs]

        aggregated_rank = self._aggregate_rank(rank_lists)[:k_dev]
        logger.info(f"Recommended developers by ranking aggregation: {aggregated_rank}")

        recommendations = {
            "predicted_components": predicted_components_name,
            "comp_prediction_score": comp_prediction_score,
            "predicted_developers": predicted_developers_name[:k_dev],
            "dev_prediction_score": dev_prediction_score,
            "similar_devs": dev_predictions_by_similarity,
            "combined_ranking": aggregated_rank,
        }

        return recommendations

    def _aggregate_rank(self, rank_lists):
        borda_scores = defaultdict(int)

        for rank_list in rank_lists:
            # Assign Borda scores to items based on their rank in each list
            for i, item in enumerate(rank_list):
                borda_scores[item] += len(rank_list) - i

        sorted_items = sorted(borda_scores.items(), key=lambda x: x[1], reverse=True)

        return [item[0] for item in sorted_items]

    def _get_recommendation_from_dev_model(self, issue, k):
        self._developer_prediction_model.eval()
        with torch.no_grad():
            tokenized_issue = self._developer_prediction_model.tokenize_text(issue)
            predictions = self._developer_prediction_model(tokenized_issue)

        output = torch.sum(torch.stack(predictions), 0)

        prediction_score, predicted_devs = output.topk(k, 1, True, True)

        predicted_devs = predicted_devs.squeeze(dim=0).cpu().numpy().tolist()
        prediction_score = prediction_score.squeeze(dim=0).cpu().numpy().tolist()
        return prediction_score, predicted_devs

    def _get_predicted_components(self, issue, k):
        self._component_prediction_model.eval()
        with torch.no_grad():
            tokenized_issue = self._component_prediction_model.tokenize_text(issue)
            predictions = self._component_prediction_model(tokenized_issue)

        output = torch.sum(torch.stack(predictions), 0)

        prediction_score, predicted_components = output.topk(k, 1, True, True)

        predicted_components = (
            predicted_components.squeeze(dim=0).cpu().numpy().tolist()
        )
        prediction_score = prediction_score.squeeze(dim=0).cpu().numpy().tolist()
        return prediction_score, predicted_components

    def _get_recommendation_by_similarity(
        self, issue, predicted_components, k_dev, k_issue, similarity_threshold
    ):
        similar_issues = self._get_top_k_similar_issues(
            issue, k=k_issue, threshold=similarity_threshold
        )
        historical_contribution = self._get_historical_contributors(
            similar_issues=similar_issues, predicted_component_ids=predicted_components
        )

        logger.debug(historical_contribution)

        top_k_devs = historical_contribution[:k_dev]

        return top_k_devs

    def _get_historical_contributors(self, similar_issues, predicted_component_ids):
        user_contribution_counts = {}
        skipped_users = set()

        for issue_index, sim_score in similar_issues:
            base_points = sim_score

            issue = self._train_data.iloc[issue_index]

            if self._component2id_map.get(issue.component) not in predicted_component_ids:
                logger.warning(
                    f"Skipping issue as label id {self._component2id_map.get(issue.component)} did not match with any of {predicted_component_ids}"
                )
                continue

            issue_number = issue.issue_number
            contributors = self._get_contribution_data(issue_number)

            for key, users in contributors.items():
                for user in users:
                    # if user not in self._expected_developers:
                    #     skipped_users.add(user)
                    #     continue

                    # if user in self._component_developers_map.get(issue.component):
                    #     user_contribution_counts[user] = (
                    #         user_contribution_counts.get(user, 0) + base_points * 1.25
                    #     )
                    # else:
                        
                    if key == "direct_assignment":
                        user_contribution_counts[user] = (
                        user_contribution_counts.get(user, 0) + base_points * 1.5
                    )
                        
                    if key == "commits" or key == "pull_request":
                        user_contribution_counts[user] = (
                        user_contribution_counts.get(user, 0) + base_points * 1.25
                    )
                    else:
                        user_contribution_counts[user] = (
                        user_contribution_counts.get(user, 0) + base_points
                    )

        if len(skipped_users) > 0:
            logger.warning(
                f"Skipped users: {skipped_users} because they don't exist in the ${{expected_developers}} list"
            )

        user_contribution_counts = sorted(
            user_contribution_counts.items(), key=lambda x: x[1], reverse=True
        )
        return user_contribution_counts

    def _get_contribution_data(self, issue_number):
        contributions = {}
        issue_file = f"{issue_number}.json"
        last_assignment = None

        if issue_file in self._all_issues:
            with open(os.path.join(self._issues_path, issue_file), "r") as file:
                issue = json.load(file)

                assignees = issue["assignees"]
                assignee_logins = (
                    [assignee["login"] for assignee in assignees]
                    if len(assignees) > 0
                    else []
                )

                contributions["direct_assignment"] = assignee_logins

                timeline = issue["timeline_data"]
                pull_requests = []
                commits = []
                discussion = []

                for timeline_event in timeline:
                    event = timeline_event["event"]

                    if event == "cross-referenced" and timeline_event["source"][
                        "issue"
                    ].get("pull_request", None):
                        actor = timeline_event["actor"]["login"]
                        pull_requests.append(actor)
                        pull_requests.append(actor)
                        last_assignment = actor

                    if event == "referenced" and timeline_event["commit_url"]:
                        actor = timeline_event["actor"]["login"]
                        commits.append(actor)
                        commits.append(actor)
                        last_assignment = actor

                    if event == "commented":
                        actor = timeline_event["actor"]["login"]
                        discussion.append(actor)

                contributions["direct_assignment"] = assignee_logins
                contributions["pull_request"] = pull_requests
                contributions["commits"] = commits
                contributions["discussion"] = discussion
                contributions["last_assignment"] = (
                    [last_assignment] if last_assignment else []
                )

        return contributions

    def _get_top_k_similar_issues(self, issue, k, threshold):
        test_embed = self._similarity_model.encode(issue)
        cos = util.cos_sim(test_embed, self._all_embeddings)

        topk_values, topk_indices = torch.topk(cos, k=k)
        topk_values = topk_values.cpu().numpy()[0]
        topk_indices = topk_indices.cpu().numpy()[0]

        similar_issues = []

        for idx, sim_score in zip(topk_indices, topk_values):
            if sim_score >= threshold:
                similar_issues.append([idx, sim_score])

        return similar_issues


In [62]:
trx = TriagerX(
    component_prediction_model=comp_model,
    developer_prediction_model=dev_model,
    similarity_model=similarity_model,
    issues_path="/home/mdafifal.mamun/notebooks/triagerX/data/openj9_issue_data_6_7_24",
    train_embeddings=train_embeddings_path,
    developer_id_map=lbl2idx,
    component_id_map=comp_lbl2id,
    train_data=df_train,
    device="cuda"
)

2024-06-13 15:40:24.829 | DEBUG    | __main__:__init__:45 - Using device: cuda
2024-06-13 15:40:24.831 | DEBUG    | __main__:__init__:46 - Loading embedding for existing issues...


In [63]:
def get_recommendation(test_idx):
    test_data = df_test.iloc[test_idx]

    print("Issue URL: ", test_data["issue_url"])
    print("Actual owner: ", test_data["owner"])
    print("Actual component: ", test_data["component"])

    return trx.get_recommendation(
        test_data.text,
        k_comp=3,
        k_dev=5,
        k_rank=10,
        similarity_threshold=0.1
    )

In [65]:
get_recommendation(315)

2024-06-13 15:40:37.626 | DEBUG    | __main__:get_recommendation:50 - Prediciting components...
2024-06-13 15:40:37.673 | INFO     | __main__:get_recommendation:57 - Predicted components: ['comp:vm', 'comp:jclextensions', 'comp:gc']
2024-06-13 15:40:37.674 | DEBUG    | __main__:get_recommendation:59 - Generating developer recommendation...
2024-06-13 15:40:37.719 | INFO     | __main__:get_recommendation:66 - Recommended developers: ['keithc-ca', 'dsouzai', 'pdbain-ibm', 'gacholio', 'DanHeidinga', 'pshipton', 'JasonFengJ9', 'fjeremic', 'tajila', 'liqunl']
2024-06-13 15:40:37.721 | DEBUG    | __main__:get_recommendation:68 - Generating recommendation by similarity...
2024-06-13 15:40:37.738 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [5, 2, 1]
2024-06-13 15:40:37.742 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id 0 did not match with any of [5, 2, 1]
2024-06-13 15:40:37.745 | WARNI

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/17785
Actual owner:  theresa-m
Actual component:  comp:vm


{'predicted_components': ['comp:vm', 'comp:jclextensions', 'comp:gc'],
 'comp_prediction_score': [13.936136245727539,
  -0.2032870054244995,
  -0.5233215093612671],
 'predicted_developers': ['keithc-ca',
  'dsouzai',
  'pdbain-ibm',
  'gacholio',
  'DanHeidinga'],
 'dev_prediction_score': [15.200265884399414,
  6.899694442749023,
  6.3180060386657715,
  6.174880027770996,
  5.672318458557129,
  5.514228820800781,
  5.337607383728027,
  4.442630767822266,
  3.694455862045288,
  1.0472722053527832],
 'similar_devs': [('pdbain-ibm', 15.576168477535248),
  ('EricYangIBM', 10.567042350769043),
  ('babsingh', 8.96440652012825),
  ('mpirvu', 7.288835823535919),
  ('pshipton', 6.412422925233841),
  ('DanHeidinga', 2.6751447916030884),
  ('dsouzai', 2.3634258806705475),
  ('gacholio', 1.364533245563507),
  ('youurayy', 1.3431364297866821),
  ('gkorland', 1.3431364297866821)],
 'combined_ranking': ['pdbain-ibm',
  'dsouzai',
  'DanHeidinga',
  'pshipton',
  'keithc-ca']}

In [86]:
"babsingh" in recommendations[678]["combined_ranking"]

False

In [114]:
import logging

logging.getLogger().setLevel(logging.DEBUG)

In [66]:
recommendations = []

for i in range(len(df_test)):
    rec = get_recommendation(i)
    recommendations.append(rec)

2024-06-13 15:41:46.213 | DEBUG    | __main__:get_recommendation:50 - Prediciting components...
2024-06-13 15:41:46.272 | INFO     | __main__:get_recommendation:57 - Predicted components: ['comp:build', 'comp:vm', 'comp:jclextensions']
2024-06-13 15:41:46.273 | DEBUG    | __main__:get_recommendation:59 - Generating developer recommendation...
2024-06-13 15:41:46.327 | INFO     | __main__:get_recommendation:66 - Recommended developers: ['keithc-ca', 'jdekonin', 'AdamBrousseau', 'pshipton', 'hangshao0', 'JasonFengJ9', 'dsouzai', 'llxia', 'Mesbah-Alam', 'dnakamura']
2024-06-13 15:41:46.329 | DEBUG    | __main__:get_recommendation:68 - Generating recommendation by similarity...
2024-06-13 15:41:46.362 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [0, 5, 2]
2024-06-13 15:41:46.364 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id 4 did not match with any of [0, 5, 2]
2024-06-13 15:41:46.36

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/16138
Actual owner:  AdamBrousseau
Actual component:  comp:infra
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/16141
Actual owner:  JasonFengJ9
Actual component:  comp:vm


2024-06-13 15:41:46.425 | INFO     | __main__:get_recommendation:57 - Predicted components: ['comp:vm', 'comp:jclextensions', 'comp:build']
2024-06-13 15:41:46.426 | DEBUG    | __main__:get_recommendation:59 - Generating developer recommendation...
2024-06-13 15:41:46.471 | INFO     | __main__:get_recommendation:66 - Recommended developers: ['JasonFengJ9', 'keithc-ca', 'knn-k', 'pshipton', 'babsingh', 'jdekonin', 'Akira1Saitoh', 'ChengJin01', 'hangshao0', 'LinHu2016']
2024-06-13 15:41:46.473 | DEBUG    | __main__:get_recommendation:68 - Generating recommendation by similarity...
2024-06-13 15:41:46.496 | DEBUG    | __main__:_get_recommendation_by_similarity:148 - [('JasonFengJ9', 27.77641525864601), ('pshipton', 13.807792752981186), ('ChengJin01', 8.83566963672638), ('keithc-ca', 5.629174500703812), ('tajila', 0.654494047164917)]
2024-06-13 15:41:46.497 | INFO     | __main__:get_recommendation:77 - Recommended developers by issue similarity: ['JasonFengJ9', 'pshipton', 'ChengJin01', 'k

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/16159
Actual owner:  pshipton
Actual component:  comp:test
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/16161
Actual owner:  nbhuiyan
Actual component:  comp:jit


2024-06-13 15:41:46.711 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [5, 4, 1]
2024-06-13 15:41:46.724 | DEBUG    | __main__:_get_recommendation_by_similarity:148 - [('babsingh', 104.87283554673195), ('fengxue-IS', 19.960008412599564), ('llxia', 13.039632260799408), ('pshipton', 11.603139698505402), ('ChengJin01', 11.090971827507019), ('smlambert', 3.844998061656952), ('DanHeidinga', 3.808920979499817), ('tajila', 1.4787962436676025), ('JasonFengJ9', 1.4787962436676025)]
2024-06-13 15:41:46.726 | INFO     | __main__:get_recommendation:77 - Recommended developers by issue similarity: ['babsingh', 'fengxue-IS', 'llxia', 'pshipton', 'ChengJin01', 'smlambert', 'DanHeidinga', 'tajila', 'JasonFengJ9']
2024-06-13 15:41:46.727 | DEBUG    | __main__:get_recommendation:79 - Aggregating ranking...
2024-06-13 15:41:46.728 | INFO     | __main__:get_recommendation:83 - Recommended developers by ranking aggregation: ['fengxue-IS', 

...

2024-06-13 15:42:54.324 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [4, 5, 0]
2024-06-13 15:42:54.328 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [4, 5, 0]
2024-06-13 15:42:54.332 | DEBUG    | __main__:_get_recommendation_by_similarity:148 - [('babsingh', 42.14380016922951), ('pshipton', 27.829750537872314), ('pdbain-ibm', 22.615646570920944), ('hangshao0', 11.278225868940353), ('ChengJin01', 10.579652488231659), ('fengxue-IS', 7.314126223325729), ('smlambert', 6.826638847589493), ('ben-walsh', 5.914861410856247), ('tajila', 4.338025450706482), ('JasonFengJ9', 3.6061741709709167), ('dhong44', 2.422708570957184), ('LongyuZhang', 1.829628199338913), ('patkarns', 1.829628199338913), ('llxia', 1.7632754147052765), ('AidanHa', 1.7396651208400726), ('liqunl', 1.7305061221122742), ('Ailloviee', 1.7305061221122742), ('DanHeidinga', 1.3880861401557922), (

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19534
Actual owner:  JasonFengJ9
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19543
Actual owner:  pshipton
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19546
Actual owner:  nbhuiyan
Actual component:  comp:jit


2024-06-13 15:42:54.571 | INFO     | __main__:get_recommendation:57 - Predicted components: ['comp:vm', 'comp:gc', 'comp:build']
2024-06-13 15:42:54.573 | DEBUG    | __main__:get_recommendation:59 - Generating developer recommendation...
2024-06-13 15:42:54.610 | INFO     | __main__:get_recommendation:66 - Recommended developers: ['gacholio', 'JasonFengJ9', 'keithc-ca', 'dsouzai', 'mpirvu', 'a7ehuo', 'pdbain-ibm', 'fjeremic', 'harryyu1994', 'pshipton']
2024-06-13 15:42:54.611 | DEBUG    | __main__:get_recommendation:68 - Generating recommendation by similarity...
2024-06-13 15:42:54.631 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [5, 1, 0]
2024-06-13 15:42:54.634 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [5, 1, 0]
2024-06-13 15:42:54.635 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with 

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19583
Actual owner:  JasonFengJ9
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19605
Actual owner:  pshipton
Actual component:  comp:vm


2024-06-13 15:42:54.868 | WARNING  | __main__:_get_historical_contributors:164 - Skipping issue as label id None did not match with any of [0, 5, 4]
2024-06-13 15:42:54.873 | DEBUG    | __main__:_get_recommendation_by_similarity:148 - [('pshipton', 15.215878278017044), ('ChengJin01', 14.877039194107056), ('JasonFengJ9', 14.485557287931442), ('babsingh', 13.749060034751892), ('LongyuZhang', 13.554231971502304), ('fengxue-IS', 7.60682275891304), ('pdbain-ibm', 4.884531021118164), ('tajila', 4.335833013057709), ('mikezhang1234567890', 3.2657817900180817), ('Yuehan-Lin', 2.562226951122284), ('acrowthe', 2.442265510559082), ('DanHeidinga', 2.18279367685318), ('llxia', 1.8084470927715302), ('psoujany', 0.7326611876487732), ('ben-walsh', 0.7257292866706848), ('fuzzy000', 0.7190034985542297)]
2024-06-13 15:42:54.874 | INFO     | __main__:get_recommendation:77 - Recommended developers by issue similarity: ['pshipton', 'ChengJin01', 'JasonFengJ9', 'babsingh', 'LongyuZhang', 'fengxue-IS', 'pdbain

In [88]:
recommendations[0]

{'predicted_components': ['comp:build', 'comp:vm', 'comp:jclextensions'],
 'comp_prediction_score': [10.378067016601562,
  5.097261428833008,
  3.514738082885742],
 'predicted_developers': ['keithc-ca',
  'jdekonin',
  'AdamBrousseau',
  'pshipton',
  'hangshao0'],
 'dev_prediction_score': [10.204696655273438,
  8.930187225341797,
  7.252267837524414,
  6.388090133666992,
  5.62624454498291,
  4.562107563018799,
  4.282619476318359,
  4.264430046081543,
  3.5828356742858887,
  3.11716628074646],
 'similar_devs': [],
 'combined_ranking': ['keithc-ca',
  'jdekonin',
  'AdamBrousseau',
  'pshipton',
  'hangshao0']}

In [67]:
combined_total = 0
dl_total = 0

for idx in range(len(df_test)):
    actual = df_test.iloc[idx]["owner"]
    combined_recommended = recommendations[idx]["combined_ranking"]
    dl_recommended = recommendations[idx]["predicted_developers"]

    print(idx, end=" ")

    if actual in combined_recommended:
        print("Comb,", end="")
        combined_total += 1

    if actual in dl_recommended:
        print("DL")
        dl_total += 1

    print("===")

0 Comb,DL
===
1 Comb,DL
===
2 Comb,DL
===
3 ===
4 Comb,DL
===
5 Comb,DL
===
6 ===
7 Comb,DL
===
8 ===
9 Comb,DL
===
10 Comb,DL
===
11 Comb,DL
===
12 Comb,DL
===
13 Comb,DL
===
14 ===
15 Comb,DL
===
16 Comb,DL
===
17 Comb,DL
===
18 Comb,DL
===
19 Comb,DL
===
20 Comb,DL
===
21 ===
22 Comb,DL
===
23 Comb,DL
===
24 Comb,DL
===
25 Comb,DL
===
26 ===
27 Comb,DL
===
28 Comb,DL
===
29 Comb,DL
===
30 Comb,DL
===
31 Comb,DL
===
32 ===
33 Comb,DL
===
34 Comb,DL
===
35 Comb,DL
===
36 ===
37 ===
38 DL
===
39 Comb,DL
===
40 Comb,DL
===
41 Comb,DL
===
42 Comb,DL
===
43 Comb,DL
===
44 Comb,DL
===
45 Comb,DL
===
46 Comb,DL
===
47 Comb,DL
===
48 Comb,DL
===
49 Comb,DL
===
50 ===
51 Comb,DL
===
52 Comb,DL
===
53 DL
===
54 Comb,DL
===
55 Comb,DL
===
56 Comb,DL
===
57 Comb,DL
===
58 Comb,DL
===
59 Comb,DL
===
60 Comb,DL
===
61 DL
===
62 ===
63 Comb,DL
===
64 Comb,DL
===
65 Comb,DL
===
66 Comb,===
67 Comb,DL
===
68 ===
69 ===
70 Comb,DL
===
71 Comb,DL
===
72 ===
73 ===
74 DL
===
75 ===
76 Comb,DL
===
77 Com

In [68]:
dl_total/len(df_test), combined_total/len(df_test)

(0.6604095563139932, 0.658703071672355)

In [31]:
anal_id = 259

print(df_test.iloc[anal_id]["issue_url"])
df_test.iloc[anal_id], recommendations[anal_id]

https://github.com/eclipse-openj9/openj9/issues/17265


(issue_number                                                17265
 issue_title     AnnotationFormatError: java.lang.IllegalArgume...
 description     I get the error AnnotationFormatError: java.la...
 issue_url       https://github.com/eclipse-openj9/openj9/issue...
 issue_state                                                closed
 creator                                                    bvsvas
 labels                                        comp:vm, userRaised
 owner                                                   theresa-m
 component                                                 comp:vm
 text            Bug Title: AnnotationFormatError: java.lang.Il...
 owner_id                                                       39
 Name: 6917, dtype: object,
 {'predicted_components': ['comp:vm', 'comp:test', 'comp:gc'],
  'comp_prediction_score': [19.85891342163086,
   0.17377474904060364,
   0.0966646671295166],
  'predicted_developers': ['ChengJin01', 'tajila', 'mikezhang1234567890'],
  

In [32]:
xx = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9_dataset_6_7_24_most_contrib.csv")
yy = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9_dataset_6_7_24_fix.csv")

In [33]:
xx.head()

,issue_number,issue_title,issue_body,issue_url,issue_state,creator,labels,assignees,component
0,2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,https://github.com/eclipse-openj9/openj9/issues/2,closed,aarongraham9,NaN,fengxue-IS,NaN
1,3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,https://github.com/eclipse-openj9/openj9/issues/3,closed,dorrab,NaN,vsebe,NaN
2,5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,https://github.com/eclipse-openj9/openj9/issues/5,closed,r30shah,NaN,anikser,NaN
3,11,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",https://github.com/eclipse-openj9/openj9/issue...,open,mgaudet,question,tajila,NaN
4,13,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,https://github.com/eclipse-openj9/openj9/issue...,closed,rservant,"enhancement, comp:build",tajila,comp:build


In [34]:
yy.head()

,issue_number,issue_title,issue_body,issue_url,issue_state,creator,labels,assignees,component
0,2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,https://github.com/eclipse-openj9/openj9/issues/2,closed,aarongraham9,NaN,gireeshpunathil,NaN
1,3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,https://github.com/eclipse-openj9/openj9/issues/3,closed,dorrab,NaN,mpirvu,NaN
2,5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,https://github.com/eclipse-openj9/openj9/issues/5,closed,r30shah,NaN,r30shah,NaN
3,11,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",https://github.com/eclipse-openj9/openj9/issue...,open,mgaudet,question,hzongaro,NaN
4,13,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,https://github.com/eclipse-openj9/openj9/issue...,closed,rservant,"enhancement, comp:build",hzongaro,comp:build
